# Top .ky Domains Mapped

 ## Install Needed Libraries and  Grab Data 

We are going to install an NSLookup and GeoIP Python Library and grab a MaxMind lookup database. 

In [1]:
! curl -L https://git.io/GeoLite2-City.mmdb -o GeoLite2-City.mmdb 
! pip3 install geoip2
! pip3 install nslookup

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 67.0M  100 67.0M    0     0  63.0M      0  0:00:01  0:00:01 --:--:--  104M
DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
You sh

## Code

### Import Neeeded Libraries & Configurations 

In [2]:
import folium
from folium.plugins import MarkerCluster
import geoip2.database
import io
from nslookup import Nslookup
import os
import pandas as pd
import requests
import sys
import warnings
import zipfile

f = open(os.devnull, 'w')
sys.stderr = f
dns_query = Nslookup(dns_servers=["8.8.8.8"])

### Grab Latest DNS File

In [3]:
r = requests.get("http://s3-us-west-1.amazonaws.com/umbrella-static/top-1m.csv.zip")
z = zipfile.ZipFile(io.BytesIO(r.content))
z.extractall()

### Read The CSV into a dataframe

In [4]:
df = pd.read_csv ('top-1m.csv', names=['Rank', 'DNS'], header=None)
df['DNS'] = df['DNS'].astype('str')

### Remove Rows That Are Not .KY

In [12]:
topky = df[df["DNS"].str.endswith(".ky")]
topky = topky.dropna().head(500)
topky

,Rank,DNS
396634,396635,cima.ky
411539,411540,gov.ky
501514,501515,candw.ky
600603,600604,autodiscover.cima.ky
682760,682761,www.cima.ky
968413,968414,campbells.com.ky
976381,976382,www.campbells.com.ky
983395,983396,ns2.candw.ky
994340,994341,ns.candw.ky


### Perform DNS Lookups On .co.uk Domains

In [13]:
topky['ip_address'] = topky['DNS'].map(lambda host:dns_query.dns_lookup(host).answer)
topky["ip_address"] = topky["ip_address"].str[0]
topky = topky.dropna()
topky

,Rank,DNS,ip_address
396634,396635,cima.ky,3.236.13.79
411539,411540,gov.ky,184.25.237.161
501514,501515,candw.ky,209.27.52.113
682760,682761,www.cima.ky,3.236.13.79
968413,968414,campbells.com.ky,108.175.155.101
976381,976382,www.campbells.com.ky,108.175.155.101
983395,983396,ns2.candw.ky,209.27.52.56
994340,994341,ns.candw.ky,209.27.52.51


### Preform GeoIP Lookup Of IP Addresses

In [14]:
def get_latitude(ip):
    try:
        reader = geoip2.database.Reader('GeoLite2-City.mmdb')
        response = reader.city(ip)
        output = response.location.latitude
        return output
    except:
        return pd.np.nan


def get_longitude(ip):
    try:
        reader = geoip2.database.Reader('GeoLite2-City.mmdb')
        response = reader.city(ip)
        output = response.location.longitude
        return output
    except:
        return pd.np.nan

def get_country(ip):
    try:
        reader = geoip2.database.Reader('GeoLite2-City.mmdb')
        response = reader.city(ip)
        output = str(response.country.iso_code)
        return output
    except:
        return pd.np.nan

unique_ips = topky['ip_address'].unique()
unique_ips = pd.Series(unique_ips, index = unique_ips)
topky['Latitude'] = topky['ip_address'].map(unique_ips.apply(get_latitude))
topky['Longitude'] = topky['ip_address'].map(unique_ips.apply(get_longitude))
topky['Country'] = topky['ip_address'].map(unique_ips.apply(get_country))
topky = topky.dropna()
topky

,Rank,DNS,ip_address,Latitude,Longitude,Country
396634,396635,cima.ky,3.236.13.79,39.0469,-77.4903,US
411539,411540,gov.ky,184.25.237.161,32.7767,-96.8050,US
501514,501515,candw.ky,209.27.52.113,19.2773,-81.2449,KY
682760,682761,www.cima.ky,3.236.13.79,39.0469,-77.4903,US
968413,968414,campbells.com.ky,108.175.155.101,37.7510,-97.8220,US
976381,976382,www.campbells.com.ky,108.175.155.101,37.7510,-97.8220,US
983395,983396,ns2.candw.ky,209.27.52.56,19.2773,-81.2449,KY
994340,994341,ns.candw.ky,209.27.52.51,19.2773,-81.2449,KY


### Map Sites

In [25]:
map = folium.Map(location=[30, -50], tiles='cartodb positron', zoom_start=4)

mcluster = MarkerCluster().add_to(map)

for index, row in topky.iterrows():
    folium.Marker(location=[row['Latitude'],row['Longitude']], popup=(row.DNS+"\n"+row.ip_address)).add_to(mcluster)

map